In [1]:
YOUR_NAME = 'sara'

AWS_PROFILE = 'cities'

grid_size = 200

In [2]:
%load_ext autoreload

In [3]:
MAIN_PATH = "s3://wri-cities-sandbox/identifyingLandSubdivisions/data"
INPUT_PATH = f'{MAIN_PATH}/input'
CITY_INFO_PATH = f'{INPUT_PATH}/city_info'
EXTENTS_PATH = f'{CITY_INFO_PATH}/extents'
BUILDINGS_PATH = f'{INPUT_PATH}/buildings'
BLOCKS_PATH = f'{INPUT_PATH}/blocks'
ROADS_PATH = f'{INPUT_PATH}/roads'
INTERSECTIONS_PATH = f'{INPUT_PATH}/intersections'
GRIDS_PATH = f'{INPUT_PATH}/city_info/grids'
SEARCH_BUFFER_PATH = f'{INPUT_PATH}/city_info/search_buffers'
OUTPUT_PATH = f'{MAIN_PATH}/output'
OUTPUT_PATH_CSV = f'{OUTPUT_PATH}/csv'
OUTPUT_PATH_RASTER = f'{OUTPUT_PATH}/raster'
OUTPUT_PATH_PNG = f'{OUTPUT_PATH}/png'
OUTPUT_PATH_RAW = f'{OUTPUT_PATH}/raw_results'

In [4]:
# Check s3 connection using AWS_PROFILE=CitiesUserPermissionSet profile 
import boto3

session = boto3.Session(profile_name=AWS_PROFILE)
s3 = session.client('s3')

# export CitiesUserPermissionSet profile to use in the next cells
import os
os.environ['AWS_PROFILE'] = AWS_PROFILE

s3.list_buckets()

{'ResponseMetadata': {'RequestId': '046P9REZSNEXTGV9',
  'HostId': 'K2SrnrZzJ+cg2K81+tWGZQ1iohMhceldYNpVtI8SXXgX+UepAZ1NeEEWf6Ij4EyjR60u6PjGg0x1RbwRlA1OBg==',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'K2SrnrZzJ+cg2K81+tWGZQ1iohMhceldYNpVtI8SXXgX+UepAZ1NeEEWf6Ij4EyjR60u6PjGg0x1RbwRlA1OBg==',
   'x-amz-request-id': '046P9REZSNEXTGV9',
   'date': 'Wed, 14 May 2025 20:16:08 GMT',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Buckets': [{'Name': 'aft-sandbox-540362055257',
   'CreationDate': datetime.datetime(2022, 9, 13, 15, 12, 20, tzinfo=tzutc())},
  {'Name': 'amplify-citiesindicatorsapi-dev-10508-deployment',
   'CreationDate': datetime.datetime(2023, 8, 30, 5, 5, 13, tzinfo=tzutc())},
  {'Name': 'cities-heat',
   'CreationDate': datetime.datetime(2023, 6, 1, 13, 22, 1, tzinfo=tzutc())},
  {'Name': 'cities-test-sandbox',
   'CreationDate': datetime.datetime(2025, 4, 18, 19, 10, 49, tzinfo=tzutc

In [5]:
# START COILED CLIENT.

import coiled

cluster = coiled.Cluster(
    workspace="wri-cities-data",
    name=f'ils-{YOUR_NAME}',
    region="us-west-2",
    arm=True,
    worker_vm_types="r8g.xlarge",
    spot_policy="spot",
    n_workers=4,
    package_sync_ignore=["pyspark", "pypandoc"]
)
client = cluster.get_client()

print(f"Started a new Dask client on Coiled. Dashboard is available at {client.dashboard_link}")

[2025-05-14 15:16:11,067][INFO    ][coiled] Fetching latest package priorities...
[2025-05-14 15:16:11,069][INFO    ][coiled.package_sync] Resolving your local subdivisions2 Python environment...
[2025-05-14 15:16:12,206][INFO    ][coiled.package_sync] Scanning 444 conda packages...
[2025-05-14 15:16:12,214][INFO    ][coiled.package_sync] Scanning 259 python packages...
[2025-05-14 15:16:13,647][INFO    ][coiled] Running pip check...
[2025-05-14 15:16:15,555][INFO    ][coiled] Validating environment...
[2025-05-14 15:16:17,732][INFO    ][coiled] Creating wheel for ~/Documents/Identifying Land Subdivisions/identifyingLandSubdivisions...
[2025-05-14 15:16:17,963][WARNING ][coiled.package_sync] Package - libopenvino-intel-cpu-plugin, libopenvino-intel-cpu-plugin~=2025.0.0 has no install candidate for Python 3.12 linux-aarch64 on conda-forge
[2025-05-14 15:16:17,965][INFO    ][coiled] Uploading coiled_local_identifyingLandSubdivisions...
[2025-05-14 15:16:19,771][INFO    ][coiled] Requesti

Started a new Dask client on Coiled. Dashboard is available at https://cluster-qdjsp.dask.host/TEIgUmffcQ6JR55s/status


In [8]:
import dask_geopandas as dgpd
import dask.dataframe as dd
import pandas as pd
from dask import delayed, compute, visualize
import geopandas as gpd
from dask.diagnostics import ProgressBar
from shapely.geometry import MultiLineString, LineString, Point
from shapely.ops import polygonize, nearest_points
#from shapely.geometry import Polygon, LineString, Point, MultiPolygon, MultiLineString, GeometryCollection
from scipy.optimize import fminbound, minimize
#from unused_code.metrics_groupby import metrics
from dask import delayed
import dask_geopandas as dgpd
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely import wkb
from scipy.stats import entropy
import time
from dask import compute

from pre_processing import *
from auxiliary_functions import *
from standardize_metrics import *
from metrics_calculation import *

import time

YOUR_NAME = 'sara'
grid_size = 200

In [ ]:
# CALCULATE BUILDING DISTANCES TO ROADS AND PRODUCE BLOCKS.

start_time = time.time()  # Start the timer

%autoreload
#from main import *

def preprocess_all_cities(city_list):
    #delayed_jobs = [delayed(calculate_building_distances_to_roads)(city) for city in city_list]
    #delayed_jobs.append([delayed(produce_blocks)(city) for city in city_list])
    delayed_jobs = []
    delayed_jobs.append([calculate_building_distances_to_roads(city) for city in city_list])
    delayed_jobs.append([produce_blocks(city,YOUR_NAME,grid_size) for city in city_list])
    results = compute(*delayed_jobs)
    return results

preprocess_all_cities(["Abidjan", "Accra", "Bamako", "Belo_Horizonte", "Bogota", "Campinas", "Cape_Town", "Lagos","Luanda", "Nairobi","Medellin"]) 
#["Abidjan", "Accra", "Nairobi", "Bamako", "Belo_Horizonte", "Bogota", "Campinas", "Cape_Town", "Lagos","Luanda", "Nairobi", "Medellin"])

end_time = time.time()  # End the timer
elapsed_time = end_time - start_time

print(f"Tasks completed in {elapsed_time:.2f} seconds.")

In [ ]:
# PRODUCE AZIMUTHS

start_time = time.time()  # Start the timer


city_list = cities = ["Abidjan", "Accra", "Bamako", "Belo_Horizonte", "Bogota", "Campinas", "Cape_Town", "Lagos","Luanda", "Nairobi", "Medellin"]
#["Abidjan", "Accra", "Nairobi", "Bamako", "Belo_Horizonte", "Bogota", "Campinas", "Cape_Town", "Lagos","Luanda", "Nairobi", "Medellin"])

delayed_jobs = []
delayed_jobs.append([produce_azimuths(city, YOUR_NAME, grid_size) for city in city_list])
results = compute(*delayed_jobs)

end_time = time.time()  # End the timer
elapsed_time = end_time - start_time

print(f"Tasks completed in {elapsed_time:.2f} seconds.")

In [9]:
start_time = time.time()  # Start the timer

cities = ["Nairobi", "Medellin"]#"Accra", "Bamako", "Belo_Horizonte", "Bogota","Campinas", "Cape_Town","Abidjan","Luanda",  ]
#cities = ['Medellin']
#"Lagos"
cities = [city.replace(' ', '_') for city in cities]

tasks = []
for city in cities:
    tasks.append(building_and_intersection_metrics(city,grid_size,YOUR_NAME))
    tasks.append(building_distance_metrics(city, grid_size, YOUR_NAME))
    tasks.append(compute_m6_m7(city, grid_size, YOUR_NAME))
    tasks.append(metrics_roads_intersections(city, grid_size, YOUR_NAME))

results = compute(*tasks)

end_time = time.time()  # End the timer
elapsed_time = end_time - start_time

print(f"Tasks completed in {elapsed_time:.2f} seconds.")

Tasks completed in 79.13 seconds.


In [ ]:
import dask_geopandas as dgpd
import pandas as pd
import numpy as np
from shapely import wkb

def consolidate_irregularity_index(city_name):
    # 1) Paths (same as before) …
    base = f'{OUTPUT_PATH_RASTER}/{city_name}/{city_name}_{grid_size}m'
    p0 = f'{base}_metrics_3_4_5_10_11_12_grid_{YOUR_NAME}.geoparquet'
    p1 = f'{base}_grid_{YOUR_NAME}_metrics_1_2.geoparquet'
    p2 = f'{base}_grid_{YOUR_NAME}_metrics_6_7.geoparquet' 
    p3 = f'{base}_grid_metrics_8_9_{YOUR_NAME}.geoparquet'

    # 2) Lazy-read  
    g0 = dgpd.read_parquet(p0)
    g1 = dgpd.read_parquet(p1)[['m1_std','m2_std','m1_raw','m2_raw','has_buildings']].drop(columns='geometry', errors='ignore')
    g2 = dgpd.read_parquet(p2)[['m6_std','m7_std','m6_raw','m7_raw']].drop(columns='geometry', errors='ignore')
    g3 = dgpd.read_parquet(p3)[['m8_std','m9_std','m8_raw','m9_raw']].drop(columns='geometry', errors='ignore')

    # 3) Merge (keeps geometry)
    df = (g0
          .merge(g1, left_index=True, right_index=True, how='left')
          .merge(g2, left_index=True, right_index=True, how='left')
          .merge(g3, left_index=True, right_index=True, how='left'))

    # 4) Mask
    std_cols = [f'm{i}_std' for i in range(1,13)]
    df['has_features'] = df['has_buildings'] | df['has_roads'] | df['has_intersections']
    df[std_cols] = df[std_cols].where(df['has_features'], np.nan)

    # 5) Persist & repartition
    df = df.persist().repartition(npartitions=8)

    all_metrics_columns_raw = ['m'+str(x)+'_raw' for x in range(1,13)]
    all_metrics_columns_final = ['m'+str(x)+'_final' for x in range(1,13)]
    all_metrics_columns_std = ['m'+str(x)+'_std' for x in range(1,13)]
    all_metrics_columns_zc = ['m'+str(x)+'_zc' for x in range(1,13)]
    
    # 6) Compute global scalars on standardized metrics
    means = df[all_metrics_columns_std].mean().compute()
    stds  = df[all_metrics_columns_std].std().compute()
    mins  = df[all_metrics_columns_std].min().compute()
    maxs  = df[all_metrics_columns_std].max().compute()

    # Convert those into z-mins / z-maxs and re-index to your zc_cols
    zmin = (mins - means) / stds
    zmax = (maxs - means) / stds
    zmin.index = all_metrics_columns_zc
    zmax.index = all_metrics_columns_zc

    # 7) One-pass normalize + index
    def normalize_partition(pdf, means, stds, zmin, zmax):
        # zero-center -> z DataFrame
        z = (pdf[all_metrics_columns_std] - means) / stds
        z.columns = all_metrics_columns_zc
        pdf[all_metrics_columns_zc] = z

        # min–max on z -> f DataFrame
        f = (z - zmin) / (zmax - zmin)
        f.columns = all_metrics_columns_final
        pdf[all_metrics_columns_final] = f

        groupA = [f"m{i}_final" for i in range(1,10)]        # metrics 1–9
        groupB = groupA + ["m12_final"]                      # metrics 1–9 plus 12
        road_metrics = ['m1_final','m2_final','m3_final','m5_final','m7_final']
        regularity_metrics = ['m6_final','m10_final','m12_final']

        # … after pdf[all_metrics_columns_final] = f …
        # compute two regularity indices and their NA‐counts
        pdf["regularity_index_A"] = f[groupA].mean(axis=1)
        pdf["na_count_A"] = f[groupA].isna().sum(axis=1)

        pdf["regularity_index_B"] = f[groupB].mean(axis=1)
        pdf["na_count_B"] = f[groupB].isna().sum(axis=1)

        pdf['subdivisions_index'] = f[road_metrics].mean(axis=1)
        pdf['second_stage'] = f[regularity_metrics].mean(axis=1)
        
        return pdf


    # 8) Build meta and map_partitions
    meta = df._meta.copy()
    for c in (
        all_metrics_columns_zc
      + all_metrics_columns_final
      + ['regularity_index_A','na_count_A','regularity_index_B','na_count_B','subdivisions_index','second_stage']
    ):
        meta[c] = pd.Series(dtype='float64')


    df = df.map_partitions(
        normalize_partition,
        means, stds, zmin, zmax,
        meta=meta
    )

    import geopandas as gpd
    
    out = f'{OUTPUT_PATH_RASTER}/{city_name}/{city_name}_{grid_size}m_all_metrics_combined_{YOUR_NAME}.geoparquet'
    # … after your df = df.map_partitions(...), instead of WKB+pd.to_parquet:
    pdf = df.compute()

    # rehydrate into a GeoDataFrame, borrow the original CRS:
    gdf = gpd.GeoDataFrame(pdf, geometry='geometry', crs=g0.crs)

    # write **one** parquet file, with embedded CRS and GeoArrow extension types:
    gdf.to_parquet(out, engine='pyarrow', index=False)

    '''
    # 9) Compute into pandas, then convert geometry to WKB
    pdf = df.compute()
    pdf['geometry'] = pdf.geometry.apply(lambda geom: wkb.dumps(geom, hex=False))

    # 10) Write one parquet file via pandas (supports single_file)
    
    pdf.to_parquet(out, engine='pyarrow', index=False)#single_file=True, 
    #df.to_parquet(out, engine='pyarrow', index=False)#
    '''
    return out

# Dispatch in parallel:
cities = ["Nairobi","Medellin"]#"Abidjan","Accra","Bamako","Belo_Horizonte","Bogota","Campinas","Cape_Town","Luanda",
#cities = ['Medellin']
futures = client.map(consolidate_irregularity_index, cities)
results = client.gather(futures)
print("Written outputs:", results)


In [ ]:
import geopandas as gpd
import pandas as pd

#cities =["Abidjan","Accra","Bamako","Belo_Horizonte","Bogota", "Nairobi","Campinas","Cape_Town","Luanda","Medellin"]
cities = ['Medellin']
paths = [f'{OUTPUT_PATH_RASTER}/{city}/{city}_{grid_size}m_all_metrics_combined_{YOUR_NAME}.geoparquet' 
         for city in cities]

# Read into GeoPandas and concat
gdfs = [gpd.read_parquet(path) for path in paths]
combined = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), crs=gdfs[0].crs)

# Write a single GeoPackage file
combined.to_file('all_cities_combined.gpkg', driver='GPKG')
print("Combined GeoPackage saved as all_cities_combined.gpkg")


In [ ]:
combined['m9_raw'].describe()

In [ ]:
grid_combined.to_parquet('../Medellin_consolidated_index_results.parquet')#regularity_index.describe() #[['m1_raw','m2_raw','regularity_index']]

In [ ]:
grid_combined.to_file('../Medellin_consolidated_index_results.shp')

In [ ]:
mde_grid = gpd.read_parquet('../Medellin_consolidated_index_results.parquet')

In [ ]:
grid_combined[['m1_raw','m1_std','m1_zero-centered','m1_final']].describe()

In [ ]:
grid_combined[['m2_raw','m2_std','m2_zero-centered','m2_final']].describe()

In [ ]:
grid_combined[['m8_raw','m8_std','m8_zero-centered','m8_final']].describe()

In [ ]:
mde_grid[['regularity_index']].describe()